# Using multiple GPUs
**Learning how to make use of the multi-GPU capabilities in PtyPy.**

All GPU-accelerated examples so far were using only a single GPU working on the reconstructions, but PtyPy supports multi-GPU execution for all its engines via MPI. Unfortunately, this is not compatible with Jupyter notebooks, but we can instead use the shell terminal provided within JupyterHub. 

`````{admonition} Tip 
:class: tip
When running PtyPy with a Python script in a compute environment that uses OpenMPI, e.g. as part of a cluster job, the multi-GPU capabilities of engines such as DM, RAAR or ML can be utilised with <code>mpirun -n 4 python ptypy_example.py</code>
`````

## Making a Python run script for PtyPy
Revisiting the earlier example on using [JSON/YAML Config Files](../basic_examples/01_using_yaml_or_json_config.ipynb) we have saved the parameter tree from the previous example on [Working With Large Data](./01_working_with_large_data.ipynb) into a config file named [./config/dls_i08_nanogold_spiral.yaml](./config/dls_i08_nanogold_spiral.yaml) making a few changes, i.e. switching to ```verbose_level=info``` and providing ```p.io.rfile``` for saving the output of the reconstruction. We can then generate a simple Python script

```python
with open("./ptypy_run_dls_i08_nanogold.py", "w") as f:
    f.write("""\
import ptypy
ptypy.load_ptyscan_module("hdf5_loader")
ptypy.load_gpu_engines("cuda")  
p = ptypy.utils.param_from_yaml("config/dls_i08_nanogold_spiral.yaml")
P = ptypy.core.Ptycho(p,level=5)
""")
```

which imports the relevant PtyScan and GPU acceleration modules, loads the parameter tree and runs the reconstruction.

---

In [ ]:
with open("./ptypy_run_dls_i08_nanogold.py", "w") as f:
    f.write("""\
import ptypy
ptypy.load_ptyscan_module("hdf5_loader")
ptypy.load_gpu_engines("cupy")  
p = ptypy.utils.param_from_yaml("config/dls_i08_nanogold_spiral.yaml")
P = ptypy.core.Ptycho(p,level=5)
""")

## Executing script in terminal on JupyterHub @ NERSC

Now, we need to open a new terminal (click blue "+" in top left corner of JupyterHub and then "Terminal") which should look something like this

```
benedikt@login01:/global/u2/b/benedikt/tutorials/notebooks/experimental_xray_data> 
```

We then need to load the PtyPy conda environment

```
module load python
conda activate /global/common/software/trn005/ptypy_env
```

which should make your terminal look like this

```
(ptypy_env) benedikt@login01:/global/u2/b/benedikt/tutorials/notebooks/experimental_xray_data> 
```

with the (PtyPy) environment loaded. If not already in the correct folder, we can navigate to the folder for this example 

```bash
cd $HOME/tutorials/notebooks/experimental_xray_data
```

and execute a multi-GPU reconstruction via ```srun``` using all 4 GPUs that is available on a Perlmutter node. 

```bash
srun -n 4 -c 2 --gpus-per-task=1 --gpu-bind=None python ptypy_run_dls_i08_nanogold.py
```

`````{admonition} Challenge 
:class: attention
Run the I08 nanogold example in a terminal using 1,2,3 and 4 GPUs and compare the speed. You can add <code>p.io.benchmark="all"</code> to get performance benchmark numbers written by PtyPy into a file called "benchmark.json"
`````

---

## Display reconstruction from ptyr file
Once the multi-GPU reconstruction has finished, we can switch back to this notebook and display the final output using

```python
import ptypy.utils.plot_client as pc
fig = pc.figure_from_ptyr("./recons/dls_i08_nanogold_DM_cupy_0200.ptyr")
```

---

In [ ]:
import ptypy.utils.plot_client as pc
fig = pc.figure_from_ptyr("./recons/dls_i08_nanogold_DM_cupy_0200.ptyr")